# Add extra information to DRZ auction results

Query to the open data dataset of the RDW.

It may take a while (10 min) to query all auction results.
- - - - 

### User variables


In [ ]:
Date = '2020-01' # yyyy-mm

toggle_to_opbod = False

### Modules and functions

In [2]:
import pandas as pd
import re 
# to keep api key hidden import this from sub dir
import assets.hidden_api_keys as hidden_api_keys
from time import sleep

# base url
apiurl = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json?$$app_token=' + hidden_api_keys.socrata_apptoken + '&'

def get_json_from_api(url,reg,c=0):
    
    '''Get json object from api'''
    
    import time

    c+=1
    try:
        df=pd.read_json(url + 'kenteken=' + reg.replace('-','').upper()).to_dict()
    except:
        if c > 10:
            print(url,reg)
            raise 
        else:
            print('pause 2 sec and try again!')
            time.sleep(2)
            df = get_json_from_api(url,reg,c)
    
    return df
    
# get_json_from_api(apiurl,'61-sf-FG')

### Load auction results

In [3]:
if toggle_to_opbod:
    file_name = '../../../python-nb/data/drz-data-opbod-{}.pkl'.format(Date)
else:
    file_name = '../data/drz-data-{}.pkl'.format(Date)
print(file_name)
drz = pd.read_pickle(file_name)

../data/drz-data-2020-01.pkl


### query rdw

In [4]:
# see what lots have a Dutch registration (license number).
hasReg = (~drz.Reg.isnull()) & (drz.Reg != 'onbekend') & (drz.Reg != '') & (~drz.LotType.isin([
    'Vaartuig',
    'Jetski',
    'Sloep',
    'Speedboot',
    'Vaartuig (Type onbekend)',
    'Motorvaartuig met opbouw (Pleziervaartuig)',
    'Aanhangwagen',
]))

# make a copy and add info
rdw = drz.copy()


In [5]:
def get_query_url(api_url, keys, token=hidden_api_keys.socrata_apptoken, field='kenteken'):
    '''construct query url'''
    
    # convert list to string
    id_list = ''.join(["'{}', ".format(k) for k in keys])
    id_list = id_list[0:-2] # remove trailing ', '
    q = api_url + '?$$app_token=' + token + '&$where='
    # add escaped soql
    soql = field + ' in(' + id_list + ')'
    q += urllib.parse.quote(soql)
    return q

Create list of dataframes with different api results

In [6]:
import urllib

# empty dictionary
dfs_regs = dict()
# first element is all registrations
key = 'registations'
dfs_regs[key] = rdw.loc[hasReg,['Reg']]
dfs_regs[key]['kenteken'] = dfs_regs[key].Reg.apply(lambda r: r.replace('-','').upper())
dfs_regs[key].index.name = 'lot_index'
dfs_regs[key] = dfs_regs[key].reset_index().set_index('kenteken')
display(dfs_regs[key].tail())
print(dfs_regs[key].shape)


,lot_index,Reg
kenteken,,
4VKL16,2020-1-7200,4-VKL-16
5VZJ26,2020-1-7201,5-VZJ-26
8VKX84,2020-1-7202,8-VKX-84
02LKX5,2020-1-7203,02-LKX-5
9VXT04,2020-1-7204,9-VXT-04


(190, 2)


In [7]:
# do main api first to get other possible apis
api_name = 'api_gekentekende_voertuigen'
key = re.sub('^api_','',api_name)
api_url = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json'
regs = dfs_regs['registations'].Reg.values
regs = [r.replace('-','').upper() for r in regs]
print(len(regs),'registrations in this set')

# query data base
q = get_query_url(api_url,regs)
dfs_regs[key] = pd.read_json(q)
dfs_regs[key].set_index('kenteken', inplace=True)
display(dfs_regs[key].tail())
print(dfs_regs[key].shape)

# query other available apis
for api_name in [c for c in dfs_regs['gekentekende_voertuigen'].columns if c.startswith('api')]:
    print(api_name, end=': ')
    key = re.sub('^api_','',api_name)
    for api_url in dfs_regs['gekentekende_voertuigen'][api_name].unique():
        print(api_url)
        # query the web
        q = get_query_url(api_url,regs)
        df0 = pd.read_json(q)
        # name of index
        df0.columns.name = api_name

        # query should return 'kenteken', make it the index
        if df0.shape[0] != 0:
            df0.set_index('kenteken', inplace=True)
            
        # Some apis return multiple values. Pivot around index number ("volgnummer")
        if api_name == 'api_gekentekende_voertuigen_assen':
            df0 = pd.pivot(df0, columns='as_nummer')

        elif api_name == 'api_gekentekende_voertuigen_brandstof':
            df0 = pd.pivot(df0, columns='brandstof_volgnummer')

        elif api_name == 'api_gekentekende_voertuigen_carrosserie':
            df0 = pd.pivot(df0, columns='carrosserie_volgnummer')

        elif api_name == 'api_gekentekende_voertuigen_carrosserie_specifiek':
            df0 = pd.pivot(df0, columns='carrosserie_volgnummer')

        # squeeze multi index
        one_level = [
            re.sub('^api_gekentekende_voertuigen_','',api_name) + '_' + '_'.join(
                [str(c) if type(c)==int else c for c in l]
            ) for l in df0.columns
        ]
        df0.columns = one_level

        # add to list
        display(df0.tail())
        print(df0.shape)
        dfs_regs[key]=df0

190 registrations in this set


,voertuigsoort,merk,handelsbenaming,vervaldatum_apk,datum_tenaamstelling,bruto_bpm,inrichting,aantal_zitplaatsen,eerste_kleur,tweede_kleur,...,api_gekentekende_voertuigen_carrosserie_specifiek,api_gekentekende_voertuigen_voertuigklasse,laadvermogen,aanhangwagen_middenas_geremd,maximale_constructiesnelheid_brom_snorfiets,vermogen_brom_snorfiets,europese_voertuigcategorie_toevoeging,europese_uitvoeringcategorie_toevoeging,aanhangwagen_autonoom_geremd,type_gasinstallatie
kenteken,,,,,,,,,,,,,,,,,,,,,
XJ614X,Personenauto,PEUGEOT,207,20100103.0,20190522,1088.0,hatchback,5.0,GRIJS,Niet geregistreerd,...,https://opendata.rdw.nl/resource/jhie-znh9.json,https://opendata.rdw.nl/resource/kmfi-hrps.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XN558K,Personenauto,OPEL,ASTRA,20210825.0,20191120,270.0,hatchback,5.0,GRIJS,Niet geregistreerd,...,https://opendata.rdw.nl/resource/jhie-znh9.json,https://opendata.rdw.nl/resource/kmfi-hrps.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XZXV44,Personenauto,MERCEDES-BENZ,S 320,20181010.0,20190715,25693.0,sedan,5.0,GRIJS,Niet geregistreerd,...,https://opendata.rdw.nl/resource/jhie-znh9.json,https://opendata.rdw.nl/resource/kmfi-hrps.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YJ05XR,Personenauto,BMW,318I U9,20190309.0,20191125,NaN,sedan,NaN,BLAUW,Niet geregistreerd,...,https://opendata.rdw.nl/resource/jhie-znh9.json,https://opendata.rdw.nl/resource/kmfi-hrps.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZK12GP,Personenauto,CHEVROLET,CAPRICE WAGON U9 BEFRAFENISAUTO,20190316.0,20181130,NaN,lijkwagen,2.0,ROSE,Niet geregistreerd,...,https://opendata.rdw.nl/resource/jhie-znh9.json,https://opendata.rdw.nl/resource/kmfi-hrps.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(164, 59)
api_gekentekende_voertuigen_assen: https://opendata.rdw.nl/resource/3huj-srit.json


,assen_aantal_assen_1,assen_aantal_assen_2,assen_spoorbreedte_1,assen_spoorbreedte_2,assen_wettelijk_toegestane_maximum_aslast_1,assen_wettelijk_toegestane_maximum_aslast_2,assen_technisch_toegestane_maximum_aslast_1,assen_technisch_toegestane_maximum_aslast_2,assen_plaatscode_as_1,assen_plaatscode_as_2,assen_aangedreven_as_1,assen_aangedreven_as_2,assen_hefas_1,assen_hefas_2
kenteken,,,,,,,,,,,,,,
XD410H,2,2,159.0,159.0,1450.0,1900.0,1450.0,1900.0,V,A,NaN,NaN,NaN,NaN
XJ614X,2,2,148.0,148.0,1000.0,900.0,1000.0,900.0,V,A,NaN,NaN,NaN,NaN
XN558K,2,2,155.0,157.0,975.0,895.0,975.0,895.0,V,A,NaN,NaN,NaN,NaN
XZXV44,2,2,158.0,158.0,1045.0,1310.0,1045.0,1310.0,NaN,NaN,NaN,NaN,NaN,NaN
YJ05XR,2,2,139.0,138.0,775.0,845.0,NaN,NaN,V,A,NaN,NaN,NaN,NaN


(115, 14)
api_gekentekende_voertuigen_brandstof: https://opendata.rdw.nl/resource/8ys7-d773.json


...

(115, 2)
api_gekentekende_voertuigen_carrosserie_specifiek: https://opendata.rdw.nl/resource/jhie-znh9.json


,carrosserie_specifiek_carrosserie_voertuig_nummer_code_volgnummer_1,carrosserie_specifiek_carrosseriecode_1,carrosserie_specifiek_carrosserie_voertuig_nummer_europese_omschrijving_1
kenteken,,,
5VZJ26,1,3,Gesloten opbouw
9VXT04,1,3,Gesloten opbouw
VL872R,1,3,Gesloten opbouw
VT504K,1,3,Gesloten opbouw
VV284J,1,3,Gesloten opbouw


(6, 3)
api_gekentekende_voertuigen_voertuigklasse: https://opendata.rdw.nl/resource/kmfi-hrps.json


""


(0, 0)


In [8]:
# Merge dataframes from different apis
df_regs = pd.concat(dfs_regs.values(), axis='columns', sort=False)
# add timestamp
df_regs['TimeStamp'] = pd.to_datetime('now').strftime('%Y%m%d')
# set lot id as index
df_regs.index.name = 'kenteken'
df_regs = df_regs.reset_index().set_index('lot_index')
display(df_regs.tail())
print(df_regs.shape)

,kenteken,Reg,voertuigsoort,merk,handelsbenaming,vervaldatum_apk,datum_tenaamstelling,bruto_bpm,inrichting,aantal_zitplaatsen,...,brandstof_milieuklasse_eg_goedkeuring_zwaar_1,brandstof_milieuklasse_eg_goedkeuring_zwaar_2,brandstof_uitstoot_deeltjes_zwaar_1,brandstof_uitstoot_deeltjes_zwaar_2,carrosserie_carrosserietype_1,carrosserie_type_carrosserie_europese_omschrijving_1,carrosserie_specifiek_carrosserie_voertuig_nummer_code_volgnummer_1,carrosserie_specifiek_carrosseriecode_1,carrosserie_specifiek_carrosserie_voertuig_nummer_europese_omschrijving_1,TimeStamp
lot_index,,,,,,,,,,,,,,,,,,,,,
2020-1-7200,4VKL16,4-VKL-16,Bedrijfsauto,SUZUKI,GRAND VITARA 1.9 D VAN,20200411.0,20191120.0,8637.0,gesloten opbouw,2.0,...,NaN,NaN,NaN,NaN,BB,Bestelwagen,NaN,NaN,NaN,20200112
2020-1-7201,5VZJ26,5-VZJ-26,Bedrijfsauto,OPEL,ASTRA SPORTS TOURER VAN,20200913.0,20191120.0,8537.0,gesloten opbouw,2.0,...,NaN,NaN,NaN,NaN,BB,Bestelwagen,1.0,3.0,Gesloten opbouw,20200112
2020-1-7202,8VKX84,8-VKX-84,Bedrijfsauto,SUZUKI,GRAND VITARA 1.9 D VAN,20200209.0,20191120.0,8637.0,gesloten opbouw,2.0,...,NaN,NaN,NaN,NaN,BB,Bestelwagen,NaN,NaN,NaN,20200112
2020-1-7203,02LKX5,02-LKX-5,Personenauto,OPEL,ASTRA,20191203.0,20191120.0,5924.0,MPV,5.0,...,NaN,NaN,NaN,NaN,AF,MPV,NaN,NaN,NaN,20200112
2020-1-7204,9VXT04,9-VXT-04,Bedrijfsauto,SUZUKI,GRAND VITARA,20200716.0,20191120.0,10297.0,gesloten opbouw,2.0,...,NaN,NaN,NaN,NaN,BB,Bestelwagen,1.0,3.0,Gesloten opbouw,20200112


(190, 111)


In [9]:
# empty dict
dfs_confcodes = dict()
# Conformity codes have sub-divisions. Four fields make a super key
key = 'conformity_codes'
dfs_confcodes[key] = dfs_regs['gekentekende_voertuigen'][[
    'typegoedkeuringsnummer', 
    'uitvoering', 
    'variant', 
    'volgnummer_wijziging_eu_typegoedkeuring'
]].dropna().drop_duplicates()
dfs_confcodes[key].reset_index(drop=True, inplace=True)
print(len(dfs_confcodes[key]),'conformity codes in this set')

display(dfs_confcodes[key].tail())
print(dfs_confcodes[key].shape)

131 conformity codes in this set


,typegoedkeuringsnummer,uitvoering,variant,volgnummer_wijziging_eu_typegoedkeuring
126,e1*2007/46*0217*06,FM5FM5A4056S0J27MG2NN1G2N,BBCAYDX01,0.0
127,e2*2007/46*0497*01,MJ1002612C1,CR9M,0.0
128,e2*2001/116*0340*00,C,C9HX,0.0
129,e4*2007/46*0996*11,BA1F2FBBA5,CA0FDBB12,0.0
130,e1*97/27*0099*01,TBAA250,N065A0,1.0


(131, 4)


In [10]:
# do conformity api and again get other possible apis
api_name = 'api_eeg_voertuigtypegoedkeuring'
key = re.sub('^api_','',api_name)
api_url = 'https://opendata.rdw.nl/resource/55kv-xf7m.json'

# query data base
# will not use sub-division, but long (year with century) version of conformity code
q = get_query_url(api_url, 
                  dfs_confcodes['conformity_codes'].typegoedkeuringsnummer.unique(), 
                  field='typegoedkeuringsnummer')
dfs_confcodes[key] = pd.read_json(q)
# more than one conformity code?
assert not (dfs_confcodes[key].groupby('typegoedkeuringsnummer')['typegoedkeuringsnummer'].count() > 1).any()
dfs_confcodes[key].set_index('typegoedkeuringsnummer', inplace=True)
display(dfs_confcodes[key].tail())
print(dfs_confcodes[key].shape)

,eu_type_goedkeuringssleutel,landcode_eeg_typegoedkeuring,richtlijn_nr_laatste_wijziging,eeg_basis_goedkeuringsnummer,eeg_uitbreiding_goedkeuringsnummer,eeg_typegoedkeuringsdatum,europese_typegoedkeurings_registratie_datum,eeg_ece_voertuig_categorie_bij_type,fabrikant,europese_typegoedkeuring_status,...,api_carrosserie_uitvoering_klasse,api_carrosserie_uitvoering_nummerieke_code,api_handelsbenaming_uitvoering,api_merk_uitvoering_toegestaan,api_motor_uitvoering,api_motor_uitvoering_brandstof,api_plaatsaanduiding_uitvoering,api_subcategorie_uitvoering,api_uitvoeringverbruik_per_uitgave,api_versnellingsbak_uitvoering
typegoedkeuringsnummer,,,,,,,,,,,,,,,,,,,,,
e9*2001/116*0052*06,e9*01/116*0052*06,e9,01/116,52,6,20060407,20060407,M1,SEAT S.A.,BT,...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...
e9*2001/116*0067*20,e9*01/116*0067*20,e9,01/116,67,20,20111004,20111004,M1,SEAT S.A.,BT,...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...
e9*2002/24*0329*00,e9*02/24*0329*00,e9,02/24,329,0,20090129,20090129,L1,NACIONAL MOTOR S.A.U.,BT,...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...
e9*92/61*0027*03,e9*92/61*0027*03,e9,92/61,27,3,20030116,20030116,L1,NACIONAL MOTOR S.A.U.,BT,...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...
e9*98/14*0037*00,e9*98/14*0037*00,e9,98/14,37,0,20000714,20000714,M1,SEAT S.A.,BT,...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...


(126, 27)


In [11]:
# add slightly different keys (year has no century)
dfs_confcodes['conformity_codes'] = dfs_confcodes['conformity_codes'].merge(
    dfs_confcodes['eeg_voertuigtypegoedkeuring'].eu_type_goedkeuringssleutel, 
    how='left', 
    left_on='typegoedkeuringsnummer', 
    right_index=True
)
dfs_confcodes['conformity_codes'].tail()

,typegoedkeuringsnummer,uitvoering,variant,volgnummer_wijziging_eu_typegoedkeuring,eu_type_goedkeuringssleutel
126,e1*2007/46*0217*06,FM5FM5A4056S0J27MG2NN1G2N,BBCAYDX01,0.0,e1*07/46*0217*06
127,e2*2007/46*0497*01,MJ1002612C1,CR9M,0.0,e2*07/46*0497*01
128,e2*2001/116*0340*00,C,C9HX,0.0,e2*01/116*0340*00
129,e4*2007/46*0996*11,BA1F2FBBA5,CA0FDBB12,0.0,e4*07/46*0996*11
130,e1*97/27*0099*01,TBAA250,N065A0,1.0,e1*97/27*0099*01


In [ ]:
# query other available apis
for api_name in [c for c in dfs_confcodes['eeg_voertuigtypegoedkeuring'].columns if c.startswith('api')]:
    print(api_name, end=': ')
    key = re.sub('^api_','',api_name)
    for api_url in dfs_confcodes['eeg_voertuigtypegoedkeuring'][api_name].unique():
        
        # reformat url
        M=re.search('https://opendata.rdw.nl/.*/([a-z0-9]{4}-[a-z0-9]{4})$', api_url)
        api_url = 'https://opendata.rdw.nl/resource/{}.json'.format(M[1])
        print(api_url)

#         if api_name in [
#             'api_as_gegevens_eeg_uitvoering',
#             'api_basisgegevens_eeg_uitvoering',
#             'api_carrosserie_uitvoering',
#             'api_carrosserie_uitvoering_klasse',
#             'api_carrosserie_uitvoering_nummerieke_code',            
#             'api_handelsbenaming_uitvoering',
#             'api_merk_uitvoering_toegestaan',
#             'api_motor_uitvoering',
#             'api_motor_uitvoering_brandstof',
#             'api_plaatsaanduiding_uitvoering',
#             'api_subcategorie_uitvoering',
#             'api_uitvoeringverbruik_per_uitgave',
#             'api_versnellingsbak_uitvoering',
#         ]: continue
        
        # query the web
        # unfortunately this needs to be done one by one, because conformity code is not unique                   
        df0 = pd.DataFrame()
        for ix, row in dfs_confcodes['conformity_codes'].iterrows():
            q = api_url
            q += '?$$app_token=' + hidden_api_keys.socrata_apptoken 
            q += '&{}=\'{}\''.format('eu_type_goedkeuringssleutel', urllib.parse.quote(row.eu_type_goedkeuringssleutel))
            q += '&{}=\'{}\''.format('eeg_uitvoeringscode', urllib.parse.quote(row.uitvoering))
            q += '&{}=\'{}\''.format('eeg_variantcode', urllib.parse.quote(row.variant))
            q += '&{}={:.0f}'.format('uitvoering_wijzigingsnummer', row.volgnummer_wijziging_eu_typegoedkeuring)
                
            n_try = 0
            OK = False
            while (n_try < 10) & (not OK):
                try:
                    res = pd.read_json(q)
                    OK = True
                    n_try = 0
                except:
                    n_try +=1
                    sleep(10)
                if n_try == 10:
                    raise

            if len(res) == 0:
                continue

            # matching data type with 'codes' for merging
            res.eu_type_goedkeuringssleutel = res.eu_type_goedkeuringssleutel.astype(str)
            res.eeg_uitvoeringscode = res.eeg_uitvoeringscode.astype(str)
            res.eeg_variantcode = res.eeg_variantcode.astype(str)
            res.uitvoering_wijzigingsnummer = res.uitvoering_wijzigingsnummer.astype(float)
            res.set_index([
                'eu_type_goedkeuringssleutel', 
                'eeg_uitvoeringscode', 
                'eeg_variantcode', 
                'uitvoering_wijzigingsnummer'
            ], inplace=True)
                
            if api_name == 'api_as_gegevens_eeg_uitvoering':               
                piv = res.pivot(columns='asnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)                
            elif api_name == 'api_handelsbenaming_uitvoering':                
                piv = res.pivot(columns='volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_carrosserie_uitvoering':                
                piv = res.pivot(columns='carrosserie_volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_plaatsaanduiding_uitvoering':
                piv = res.pivot(columns='plaats_aanduiding_volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)            
            elif api_name == 'api_uitvoeringverbruik_per_uitgave':
                piv = res.pivot(columns='uitvgavenummer_verbruikboek')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_motor_uitvoering':
                piv = res.pivot(columns='volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_versnellingsbak_uitvoering':
                piv = res.pivot(columns='volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_motor_uitvoering_brandstof':
                piv = pd.pivot_table(res, index=res.index.names, columns=['volgnummer', 'brandstof_volgnummer'])
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)                
            elif api_name == 'api_merk_uitvoering_toegestaan':
                piv = pd.pivot_table(res, index=res.index.names, values=res.columns, aggfunc=list)
                df0 = pd.concat([df0, piv], sort=False)
            else:
                assert res.shape[0] == 1
                columns = ['{}_{}'.format(re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)), c) for c in res.columns] 
                res.columns = columns
                df0 = pd.concat([df0, res], sort=False)
                
            print('.', end='')
        if len(df0)==0:
            print('No results for this api')
            continue


        # add to dict
        dfs_confcodes[key] = df0
        display(dfs_confcodes[key].tail())
        print(dfs_confcodes[key].shape)


In [13]:
# Merge dataframes from different apis

# first merge first two results
tmp = dfs_confcodes['conformity_codes'].merge(dfs_confcodes['eeg_voertuigtypegoedkeuring'], how='left', 
                                              left_on='typegoedkeuringsnummer',
                                              right_index=True
                                             )
tmp['uitvoering']
tmp.rename(columns={
    'eu_type_goedkeuringssleutel_x': 'eu_type_goedkeuringssleutel',
    'uitvoering': 'eeg_uitvoeringscode',
    'variant': 'eeg_variantcode',
    'volgnummer_wijziging_eu_typegoedkeuring': 'uitvoering_wijzigingsnummer',
}, inplace=True)
tmp.set_index(['eu_type_goedkeuringssleutel', 'eeg_uitvoeringscode', 'eeg_variantcode', 'uitvoering_wijzigingsnummer'], inplace=True)

# merge with subsequent api results
df_confcodes = pd.concat([tmp] + list(dfs_confcodes.values())[2:], axis='columns', sort=False)
# add timestamp
df_confcodes['TimeStamp'] = pd.to_datetime('now').strftime('%Y%m%d')

display(df_confcodes.tail())
print(df_confcodes.shape)

,,,,typegoedkeuringsnummer,eu_type_goedkeuringssleutel_y,landcode_eeg_typegoedkeuring,richtlijn_nr_laatste_wijziging,eeg_basis_goedkeuringsnummer,eeg_uitbreiding_goedkeuringsnummer,eeg_typegoedkeuringsdatum,europese_typegoedkeurings_registratie_datum,eeg_ece_voertuig_categorie_bij_type,fabrikant,...,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_2,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_3,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_4,versnellingsbak_uitvoering_type_versnellingsbak_1,versnellingsbak_uitvoering_aantal_versnellingen_ondergrens_1,versnellingsbak_uitvoering_aantal_versnellingen_bovengrens_1,versnellingsbak_uitvoering_type_versnellingsbak_2,versnellingsbak_uitvoering_aantal_versnellingen_ondergrens_2,versnellingsbak_uitvoering_aantal_versnellingen_bovengrens_2,TimeStamp
eu_type_goedkeuringssleutel,eeg_uitvoeringscode,eeg_variantcode,uitvoering_wijzigingsnummer,,,,,,,,,,,,,,,,,,,,,
e9*01/116*0052*06,SGFM5AF013M0GG,AFBSEX01,0.0,e9*2001/116*0052*06,e9*01/116*0052*06,e9,01/116,52.0,6.0,20060407.0,20060407.0,M1,SEAT S.A.,...,NaN,NaN,NaN,H,5.0,5.0,NaN,NaN,NaN,20200112
e9*01/116*0067*20,SGEFM52R0317MGGI,SCCFWAX0,0.0,e9*2001/116*0067*20,e9*01/116*0067*20,e9,01/116,67.0,20.0,20111004.0,20111004.0,M1,SEAT S.A.,...,NaN,NaN,NaN,H,5.0,5.0,NaN,NaN,NaN,20200112
e9*02/24*0329*00,1A,3C,0.0,e9*2002/24*0329*00,e9*02/24*0329*00,e9,02/24,329.0,0.0,20090129.0,20090129.0,L1,NACIONAL MOTOR S.A.U.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200112
e9*92/61*0027*03,B,1E,0.0,e9*92/61*0027*03,e9*92/61*0027*03,e9,92/61,27.0,3.0,20030116.0,20030116.0,L1,NACIONAL MOTOR S.A.U.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200112
e9*98/14*0037*00,SGFM5850281N1I4,SCAUDX01,0.0,e9*98/14*0037*00,e9*98/14*0037*00,e9,98/14,37.0,0.0,20000714.0,20000714.0,M1,SEAT S.A.,...,C,C,C,H,5.0,5.0,NaN,NaN,NaN,20200112


(153, 404)


In [ ]:
# Merge confirmation codes with registrations
df_regs.index.name = 'lot_index'
df = df_regs.reset_index().merge(df_confcodes.reset_index(), how='left',
                   left_on=['typegoedkeuringsnummer', 'uitvoering', 'variant', 'volgnummer_wijziging_eu_typegoedkeuring'],
                   right_on=['typegoedkeuringsnummer', 'eeg_uitvoeringscode', 'eeg_variantcode', 'uitvoering_wijzigingsnummer'],
).set_index('lot_index')
display(df.tail(10))
print(df.shape)
assert all(df.columns.value_counts() == 1)

# Merge rdw and drz

In [15]:
rdw = pd.concat([rdw, df.add_prefix('rdw_')], axis='columns', sort=False)
# There should be no duplicates in column names
assert all(rdw.columns.value_counts() == 1)
# indices should match
assert rdw.index.isin(drz.index).all() & drz.index.isin(rdw.index).all()

### Saving

In [16]:
if toggle_to_opbod:
    file_name = '../../../python-nb/data/rdw-data-opbod-{}.pkl'.format(Date)
else:
    file_name = '../data/rdw-data-{}.pkl'.format(Date)
print(file_name)
rdw.to_pickle(file_name)

../data/rdw-data-2020-01.pkl
